In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pylab as py
import pandas as pd
import requests
import urllib
import urllib.request
import html.parser
import requests
from selenium import webdriver
driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")

In [3]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

In [4]:
# match indexing for EPL url
i_18_19 = [38308, 38687]
i_17_18 = [22342, 22721]
i_16_17 = [14040, 14419]
i_15_16 = [12115, 12494]
i_14_15 = [9611, 9990]
i_13_14 = [9231, 9610]
i_12_13 = [7864, 8243]
i_11_12 = [7467, 7846]

In [7]:
# import html from url
print('yes')
url = "https://www.premierleague.com/match/38678"
stats_url = 'https://datahub.io/sports-data/english-premier-league/datapackage.json'
team_url='https://www.whoscored.com/Teams/26/Archive/England-Liverpool'
opener = AppURLopener()
source_code = requests.get(url)
team_text=driver.get(team_url)
#print(team_text)
soup_team = BeautifulSoup(driver.page_source, 'lxml')
#print(soup_team)
'''
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)
'''
plain_text = source_code.text
soup = BeautifulSoup(plain_text, 'html.parser')
#print(soup)




yes


C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  


In [8]:
# generate data_ppp: data from EPL website containing (p)layer name, (p)osition, and (p)layed time
role = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
data_ppp = soup.find_all('div', class_='matchLineupTeamContainer')
match_stats=soup.find_all('div', class_='matchLineupTeamContainer')
#print((data_ppp))




In [9]:
M_ppp = [[], []] # matrix for home & away 'ppp' information
for team_num in range(2): # iterate over home & away teams
    team_i = data_ppp[team_num].getText().split('\n')
    team_i = [element.strip() for element in team_i if (element.strip() != '' and element.strip() != 'C')]
    # team_i now is a list: [..."p_i uniform #", "p_i name", "p_i position", "##'; subbed in/out time",
    #                        ... "Substitues", "p_sub_i uniform #", ...]
    i_sub = team_i.index('Substitutes') # index showing 'Substitues' header
    team_i_11, team_i_sub = team_i[:i_sub], team_i[i_sub+1:] # distinguish between starting 11 and sub's
    # iterate over starting 11 players
    i_role = [0] + [i for i in range(len(team_i_11)) if team_i_11[i] in role]
    for i in range(len(i_role)-1):
        if i == 0: L_i = team_i_11[i_role[i]+1: i_role[i+1]+1]
        else: L_i = team_i_11[i_role[i]+2: i_role[i+1]+1]
        if len(L_i) == 2: # played full time
            M_ppp[team_num].append(L_i + [1.0])
        else: # did not play full time
            M_ppp[team_num].append([L_i[0], L_i[2], int(L_i[1][:-1])/90.0])
    # iterate over substitute players
    i_role = [0] + [i for i in range(len(team_i_sub)) if team_i_sub[i] in role]
    for i in range(len(i_role)-1):
        if i == 0: L_i = team_i_sub[i_role[i]+1: i_role[i+1]+1]
        else: L_i = team_i_sub[i_role[i]+2: i_role[i+1]+1]
        if len(L_i) == 3: # sub player played
            M_ppp[team_num].append([L_i[0], L_i[2], (90.0-int(L_i[1][:-1]))/90.0])
H_ppp, A_ppp = np.array(M_ppp[0]), np.array(M_ppp[1])
print(H_ppp)
#print(A_ppp)

[['Mat Ryan' 'Goalkeeper' '1.0']
 ['Lewis Dunk' 'Defender' '1.0']
 ['Bruno' 'Defender' '0.9333333333333333']
 ['Bernardo' 'Defender' '1.0']
 ['Shane Duffy' 'Defender' '1.0']
 ['Yves Bissouma' 'Midfielder' '1.0']
 ['Pascal Groß' 'Midfielder' '1.0']
 ['Alireza Jahanbakhsh' 'Midfielder' '0.7444444444444445']
 ['Beram Kayal' 'Midfielder' '1.0']
 ['Anthony Knockaert' 'Midfielder' '1.0']
 ['Glenn Murray' 'Forward' '0.7444444444444445']
 ['Martín Montoya' 'Defender' '0.06666666666666667']
 ['Florin Andone' 'Forward' '0.25555555555555554']
 ['Jürgen Locadia' 'Forward' '0.25555555555555554']]


In [10]:

#print(soup_team)


In [51]:
import re
data_past_seasons=soup_team.find_all('option',string=re.compile("Premier"))
tag_lister=[]
i=0
for option in data_past_seasons:
    tag_lister.append(option['value'])
    i+=1
#tag_lister=[]
#for i in range(len(data_past_seasons)):
#   value=data_past_seasons.getText()
    

#season_values=data_past_seasons.count('Premier')
print(tag_lister)



['16368', '15151', '13796', '12496', '9155', '7794', '6531', '5476', '4345', '3115']


In [62]:
team_history_front='https://www.whoscored.com/Teams/26/Archive/England-Liverpool?stageId='
for i in range (1):
    driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
    history_url=team_history_front+tag_lister[i]
    historical= driver.get(history_url) 
    soup_historical = BeautifulSoup(driver.page_source, 'lxml')
    #print(soup_historical)
    table=soup_historical.find_all('tbody', id="player-table-statistics-body")
    print(table)
    driver.close()

[<tbody id="player-table-statistics-body"><tr class=""><td style="vertical-align: top">	1</td> <td style="vertical-align: top"><span class="ui-icon country flg-eg"></span> </td><td class="pn"><a class="player-link" href="/Players/108226/Show/Mohamed-Salah">Mohamed Salah </a><a class="player-meta-data" href="/Teams/26/Show/England-Liverpool"><span class="team-name">Liverpool, </span></a><span class="player-meta-data">27</span><span class="player-meta-data">,  AM(CLR),FW  </span></td> <td>175</td> <td>71</td> <td>37(1)</td><td class="minsPlayed">3262	</td><td class="goal">22	</td><td class="assistTotal">8	</td><td class="yellowCard">1	</td><td class="redCard">-	</td><td class="shotsPerGame">3.6	</td><td class="passSuccess">75.9	</td><td class="aerialWonPerGame">0.3	</td><td class="manOfTheMatch">8	</td><td class="rating sorted">7.57</td> </tr><tr class="alt"><td style="vertical-align: top">	2</td> <td style="vertical-align: top"><span class="ui-icon country flg-nl"></span> </td><td class